In [1]:
%load_ext autoreload
%autoreload 2

import glob,time
import pylab as pl
import numpy as np
import matplotlib
import astropy.io.fits as fits
import matplotlib.pyplot as plt

from   astropy.table import Table, vstack, join, unique, hstack
from   desimodel.focalplane.geometry import xy2radec 
from   desimodel.io import load_fiberpos 
from   desitarget.geomask import circles 
from   desitarget.sv3.sv3_targetmask import desi_mask, bgs_mask, mws_mask 

import healpy as hp
import pandas as pd
import matplotlib.gridspec as gridspec
from astropy.io import fits as fits
from astropy import constants as const
from astropy import units as u
from astropy.table import QTable
from astropy.table import unique 

import math

import sys

'''
sys.path.append('/global/homes/l/lbigwood/LSS/py')
import LSS
import LSS.SV3
import LSS.SV3.cattools as cattools
'''
from desitarget.geomask import get_imaging_maskbits 
from bgsfunc import tile2rosette, get_tiles

nside = 32
orig_density_per_deg = 2500 #random

#files
sv3_randoms = '/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/random0/rancomb_brightwdup_Alltiles.fits'
main_survey_randoms = '/global/cfs/cdirs/desi/target/catalogs/dr9/0.49.0/randoms/resolve/randoms-1-0.fits'

In [2]:
tiles = get_tiles()

In [3]:
#making full sv3 ledger 
ledger = vstack([Table.read(x) for x in glob.glob('/global/cscratch1/sd/mjwilson/S4MOCK/SV3REAL/SV3REALLEDGER/bright/*')])

In [4]:
#update ledger so only have targets where initial priority is same as priority, i guess like first run 
ledger    = ledger[ledger['PRIORITY_INIT'] == ledger['PRIORITY']]

In [5]:
#list of tile ids for the rosette 15 - this is rosette we are working with for now 
tids      = tiles['TILEID'].data[tiles['ROSETTE'] == 15]
tids

array([417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427])

In [6]:
#root for fibre assign files 
root      = '/global/cscratch1/sd/mjwilson/S4MOCK/SV3REAL/SV3REALASSIGN/'

In [7]:
#list of all the sv3 nights so far i think
nights    = [x.split('/')[-1] for x in sorted(glob.glob(root + '/*'))]
nights.remove('SV3.ecsv')
nights

['20210405',
 '20210407',
 '20210408',
 '20210409',
 '20210410',
 '20210411',
 '20210412',
 '20210413',
 '20210414',
 '20210415',
 '20210416',
 '20210417',
 '20210418',
 '20210419',
 '20210420',
 '20210422',
 '20210429',
 '20210430',
 '20210501',
 '20210502',
 '20210503',
 '20210504',
 '20210505',
 '20210506',
 '20210507',
 '20210508',
 '20210509',
 '20210510',
 '20210511',
 '20210512',
 '20210513',
 '20210514',
 '20210518',
 '20210521',
 '20210529']

In [8]:
datamodel = np.array([], dtype=[ 

    ('TILEID', '>i8'), ('LOCATION', '>i8'), ('TARGETID', '>i8'), 

    ('EXPID', '>f4') ]) 

 
t = Table(datamodel) 
# We'll use vstack of a list of interim tables.

f = fits.open('/global/cfs/cdirs/desi/spectro/redux/everest/exposures-everest.fits')
exposures_file = Table(f[1].data)

exposures = exposures_file['TILEID', 'EXPID']

#building full table based on individual tiles 
result = []


for night in nights:
    fbas      = sorted(glob.glob(root + '/' + night + '/fba-*.fits'))
    ts        = np.array([x.split('/')[-1].replace('.fits','').replace('fba-','') for x in fbas]).astype(np.int64)
    
    ts        = ts[np.isin(ts, tids)]
    
    print(night, ts)
    

    for i, tid in enumerate(ts):
        
        fpath = root + '/' + night + '/fba-{:06d}.fits'.format(tid)
        
        fba   = Table.read(fpath, hdu='FASSIGN')
        
        fba   = join(fba,  ledger, keys='TARGETID', join_type='left')
    
        # **  New  **: do the same for fba, this removes targets not in the (main survey, science) ledger.   
        fba   = fba[~fba['RA'].mask]

        # keep good fibres only for fba too 
        fba   = fba[fba['FIBERSTATUS'] == 0]
        
        # Keep only DESI targets
        fba   = fba[(fba['SV3_DESI_TARGET'].data & desi_mask['BGS_ANY']) != 0] 
        
        #we want table of targetid, location, fiber, tileid,expid
        #we set up subtable for first three
        subtable = fba['TARGETID', 'LOCATION', 'FIBER']
        
        #then add column which is the tileid repeated over the whole length of the table 
        subtable['TILEID'] = np.array([tid] * len(subtable))

        #we then find exposures for a given tile using the tielid file 
        tile_exps = exposures[np.isin(exposures['TILEID'], [tid])]
        
        all_exps = []
        
        subtable_orig = subtable
        
        #then we go through and copy the table so it is done for each exposure 
        for i in range(len(tile_exps['EXPID'].data)):
            
            exps = np.array([tile_exps['EXPID'].data[i]]* len(subtable_orig))
            all_exps.append(exps)
            
        subtable = [subtable]*len(tile_exps['EXPID'].data)
        subtable = vstack(subtable)
        
        subtable['EXPID'] = np.concatenate(all_exps)
        
            
        #and then we add this to total table as this was just for one tile 
        result.append(subtable)
            
        
        
assignment_database = vstack(result)

        
assignment_database.rename_column('TARGETID', 'REALASSIGNED_TARGETID')    

20210405 [417]


20210407 []
20210408 []
20210409 []
20210410 [418]
20210411 [419]
20210412 []
20210413 []
20210414 []
20210415 [420]
20210416 []
20210417 [421]
20210418 []
20210419 []
20210420 [422]
20210422 [423]
20210429 [424]
20210430 []
20210501 [425]
20210502 []
20210503 []
20210504 []
20210505 []
20210506 []
20210507 []
20210508 [426]
20210509 []
20210510 []
20210511 []
20210512 []
20210513 []
20210514 []
20210518 []
20210521 []
20210529 [427]


In [9]:
unique(assignment_database['TILEID','EXPID'])

TILEID,EXPID
int64,int32
417,84123
418,84245
419,84827
419,84828
420,85106
421,85518
422,85642
423,86513
424,86755


In [10]:
U = unique(assignment_database['TILEID', 'EXPID'])

#use '{:08d}'.format(123) when doing it generally
#and can use ! to use terminal 

tsnr_table = []

for i in range(len(U)):
    
    for p in range(0,10):
        tileid = U['TILEID'][i]
        expid = U['EXPID'][i]
        petal = p
        
        rules = [tileid == 425,expid == 87127,p == 5]

        
        if all(rules):
            continue 
        
        
        coadd_file = fits.open('/global/cfs/cdirs/desi/spectro/redux/everest/tiles/perexp/{}/{:08d}/coadd-{}-{}-exp{:08d}.fits'.format(U['TILEID'][i],U['EXPID'][i],p,U['TILEID'][i],U['EXPID'][i]))

        subtable = Table(coadd_file['FIBERMAP'].data)['LOCATION','TARGETID','COADD_FIBERSTATUS']

        subtable['TILEID'] = np.array([tileid]*len(coadd_file['FIBERMAP'].data))
        subtable['EXPID'] = np.array([expid]*len(coadd_file['FIBERMAP'].data))
        subtable['PETAL'] = np.array([p]*len(coadd_file['FIBERMAP'].data))

        subtsnr = Table(coadd_file['SCORES'].data)['TSNR2_BGS']

        assert  (len(subtsnr) == len(subtable))

        x = Table(coadd_file['SCORES'].data)['TSNR2_BGS', 'TARGETID']
        x.rename_column('TARGETID', '_TID')

        subtable['TSNR2_BGS'] = x['TSNR2_BGS']
        # subtable['_TID'] = x['_TID']

        subtable.rename_column('TARGETID', 'TSNR_TARGETID')

        tsnr_table.append(subtable)

        
tsnr_table=vstack(tsnr_table)
        


In [12]:
tsnr_table

LOCATION,TSNR_TARGETID,COADD_FIBERSTATUS,TILEID,EXPID,PETAL,TSNR2_BGS
int64,int64,int32,int64,int32,int64,float32
311,39633441209188942,0,417,84123,0,1346.9923
272,39633443784491294,0,417,84123,0,1514.8828
252,39633443784493049,0,417,84123,0,1442.1736
156,39633446334629677,0,417,84123,0,1442.7393
198,39633443788686693,0,417,84123,0,1562.8406
204,616094196087915290,0,417,84123,0,1624.0398
233,39633443784494131,0,417,84123,0,1518.8994
172,39633443784494248,0,417,84123,0,1399.8635
310,616094193512612879,0,417,84123,0,1619.4718


In [13]:
S4MOCK_database = join(assignment_database, tsnr_table,keys=['TILEID', 'LOCATION', 'EXPID'], join_type='left') 

In [14]:
S4MOCK_database_targetid = S4MOCK_database.group_by('REALASSIGNED_TARGETID') 
 
result = S4MOCK_database_targetid.groups.aggregate(np.add)   

In [116]:
result

REALASSIGNED_TARGETID,LOCATION,FIBER,TILEID,EXPID
int64,int64,int64,int64,int64
39633435978895151,2832,1020,838,169655
39633435983089279,1050,796,838,169655
39633435983089911,1048,712,838,169655
39633435987280232,2100,1592,1700,348643
39633435987280309,522,478,427,90249
39633435987281447,1536,853,1265,259904
39633435987281579,1044,956,838,169655
39633435987282090,1042,918,838,169655
39633435987282257,518,470,421,85518


In [ ]:
#extras as backup
"""
datamodel = np.array([], dtype=[ 

    ('TILEID', '>i8'), ('LOCATION', '>i8'), ('TARGETID', '>i8'), 

    ('EXPID', '>f4') ]) 

 
t = Table(datamodel) 
# We'll use vstack of a list of interim tables.

f = fits.open('/global/cfs/cdirs/desi/spectro/redux/everest/exposures-everest.fits')
exposures_file = Table(f[1].data)

exposures = exposures_file['TILEID', 'EXPID']

result = []

for night in nights:
    fbas      = sorted(glob.glob(root + '/' + night + '/fba-*.fits'))
    ts        = np.array([x.split('/')[-1].replace('.fits','').replace('fba-','') for x in fbas]).astype(np.int64)
    
    ts        = ts[np.isin(ts, tids)]
    
    print(night, ts)
    

    for i, tid in enumerate(ts):
        print(tid)
        fpath = root + '/' + night + '/fba-{:06d}.fits'.format(tid)
        tpath = root + '/' + night + '/{:06d}-targ.fits'.format(tid)

        tra   = fits.open(fpath)[1].header['TILERA']
        tdec  = fits.open(fpath)[1].header['TILEDEC']
        
        #not sure why we open targ as don't use it 
        targ  = Table.read(tpath)
        fba   = Table.read(fpath, hdu='FASSIGN')
        ftarg = Table.read(fpath, hdu='FTARGETS')
        #favl  = Table.read(fpath, hdu='FAVAIL')
    
        # Only keep targets available to a good fiber. 
        #good_fib = (fba['FIBERSTATUS']==0)
        #favl  = favl[np.isin(favl['LOCATION'], fba['LOCATION'].data[good_fib])] 
    
        #favl  = join(favl, ledger, keys='TARGETID', join_type='left')
        fba   = join(fba,  ledger, keys='TARGETID', join_type='left')
        
        # ** Check if there's no match found in the ledger (which leads to a 'masked' value for RA) ** 
        #favl  = favl[~favl['RA'].mask]

        # **  New  **: do the same for fba, this removes targets not in the (main survey, science) ledger.   
        fba   = fba[~fba['RA'].mask]

        # keep good fibres only for fba too (did it for favl above)
        fba   = fba[fba['FIBERSTATUS'] == 0]

        #gloc  = fba['LOCATION']
        
        # Keep only DESI targets
        fba   = fba[(fba['SV3_DESI_TARGET'].data & desi_mask['BGS_ANY']) != 0] 
        
        subtable = fba['TARGETID', 'LOCATION', 'FIBER']
        subtable['TILEID'] = np.array([tid] * len(subtable))

        tile_exps = exposures[np.isin(exposures['TILEID'], [tid])]
        
        repeats = []
        
        for expid in tile_exps['EXPID'].data:
            subtable['EXPID'] = np.array([expid] * len(subtable))
            
            repeats.append(subtable)

        repeats = vstack(repeats)
            
        result.append(repeats)
            
        
        
assignment_database = vstack(result)
        
assignment_database.rename_column('TARGETID', 'REALASSIGNED_TARGETID')  
"""